In [16]:
64 * 6

384

In [17]:
3800 / 6

633.3333333333334

In [14]:
len("ATTTCTCTCTCTCTCTGAGATCGATCGATCGAT") # 33 -> 32

33

In [15]:
len("ATTTCTCTCTCTCTCTGAGATCGATCGATCGATATCTCTCGAGCTAGC") # 48 -> 32

48

In [1]:
import os

os.chdir("/home/robsyc/Desktop/thesis/MB-VAE-DTI")

In [2]:
import h5torch

from mb_vae_dti.processing import load_h5torch_DTI

In [10]:
file_path = "data/processed/data.h5torch"
with h5torch.File(file_path, "r") as f:
    for key in f.keys():
        print(key)
        print(f[key].keys())
    targets_new = [s.decode("utf-8") for s in f["1/Target_AA"][:]]

test_davis_metz = load_h5torch_DTI(
    setting="split_cold",
    split="test",
    datasets=["in_DAVIS", "in_Metz", "in_BindingDB_Kd"]
)
aa = test_davis_metz[153]['1/Target_AA']
test_davis_metz[153]

# with h5torch.File(file_path, "r") as f:
#     print(f["1/Target_fp"][153][:20])
#     print(f["1/Target_fp"][153].shape)

0
<KeysViewHDF5 ['Drug_ID', 'Drug_InChIKey', 'Drug_SMILES']>
1
<KeysViewHDF5 ['Target_AA', 'Target_DNA', 'Target_Gene_name', 'Target_ID', 'Target_RefSeq_ID', 'Target_UniProt_ID']>
central
<KeysViewHDF5 ['data', 'indices']>
unstructured
<KeysViewHDF5 ['Y_KIBA', 'Y_pKd', 'Y_pKi', 'in_BindingDB_Kd', 'in_BindingDB_Ki', 'in_DAVIS', 'in_KIBA', 'in_Metz', 'split_cold', 'split_rand']>
Using boolean mask for mapping (7145 indices)
Verified alignment: all unstructured data has 396469 elements


{'central': True,
 '0/Drug_ID': 'D000028',
 '0/Drug_InChIKey': 'XZXHXSATPCNXJR-ZIADKAODSA-N',
 '0/Drug_SMILES': 'COC(=O)c1ccc2c(c1)NC(=O)C2=C(Nc1ccc(N(C)C(=O)CN2CCN(C)CC2)cc1)c1ccccc1',
 '1/Target_AA': 'MEQPPAPKSKLKKLSEDSLTKQPEEVFDVLEKLGEGSYGSVFKAIHKESGQVVAIKQVPVESDLQEIIKEISIMQQCDSPYVVKYYGSYFKNTDLWIVMEYCGAGSVSDIIRLRNKTLIEDEIATILKSTLKGLEYLHFMRKIHRDIKAGNILLNTEGHAKLADFGVAGQLTDTMAKRNTVIGTPFWMAPEVIQEIGYNCVADIWSLGITSIEMAEGKPPYADIHPMRAIFMIPTNPPPTFRKPELWSDDFTDFVKKCLVKNPEQRATATQLLQHPFIKNAKPVSILRDLITEAMEIKAKRHEEQQRELEEEEENSDEDELDSHTMVKTSVESVGTMRATSTMSEGAQTMIEHNSTMLESDLGTMVINSEDEEEEDGTMKRNATSPQVQRPSFMDYFDKQDFKNKSHENCNQNMHEPFPMSKNVFPDNWKVPQDGDFDFLKNLSLEELQMRLKALDPMMEREIEELRQRYTAKRQPILDAMDAKKRRQQNF',
 '1/Target_DNA': 'AGTAAACTAAAAAAGCTGAGTGAAGACAGTTTGACTAAGCAGCCTGAAGAAGTTTTTGATGTATTAGAGAAGCTTGGAGAAGGGTCTTATGGAAGTGTATTTAAAGCAATACACAAGGAATCCGGTCAAGTTGTCGCAATTAAACAAGTACCTGTTGAATCAGATCTTCAGGAAATAATCAAAGAAATTTCCATAATGCAGCAATGTGACAGCCCATATGTTGTAAAGTACTATGGCAGTTATTTTAAGAATACAGACCTCTGGATTGTTATGGAGTACTGTGGC

In [14]:
file_path = "data/processed/protein_embeddings_demo.h5torch"
with h5torch.File(file_path, "r") as f:
    print(f.keys())
    for key in f.keys():
        print("---")
        print(key)
        print(f[key])
    print("---")
    print(f["central"][:])

# h5torch dataset
demo_dataset = h5torch.Dataset(file_path)
sample = demo_dataset[0]
for key, value in sample.items():
    print(key)
    print(type(value))
    try:
        print(value.shape)
        print(value.dtype)
    except:
        print(len(value))
    print("---")

sample

<KeysViewHDF5 ['0', 'central']>
---
0
<HDF5 group "/0" (4 members)>
---
central
<HDF5 dataset "central": shape (3,), type "<i8">
---
[0 1 2]
central
<class 'numpy.int64'>
()
int64
---
0/esm_embeddings
<class 'numpy.ndarray'>
(2560,)
float32
---
0/espf_embeddings
<class 'numpy.ndarray'>
(4170,)
float32
---
0/protein_sequences
<class 'numpy.str_'>
()
<U189
---
0/y
<class 'numpy.float64'>
()
float64
---


{'central': 0,
 '0/esm_embeddings': array([ 0.44986314,  0.67291075, -0.4829714 , ..., -0.1495707 ,
         0.19579366,  0.83552694], dtype=float32),
 '0/espf_embeddings': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 '0/protein_sequences': 'MTEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVIDGETCLLDILDTAGQEEYSAMRDQYMRTGEGFLCVFAINNTKSFEDIHQYREQIKRVKDSDDVPMVLVGNKCDLAARTVESRQAQDLARSYGIPYIETSAKTRQGVEDAFYTLVREIRQHKLRKLNPPDESGPGCMSCKCVLS',
 '0/y': 0.918254835710806}

In [11]:
import os
import numpy as np
from pathlib import Path
import h5torch
from typing import List

from mb_vae_dti.processing.embedding import add_embeddings_to_h5torch, PROCESSED_DIR

# Example protein sequences (short ones for demonstration)
EXAMPLE_PROTEINS = [
    "MTEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVIDGETCLLDILDTAGQEEYSAMRDQYMRTGEGFLCVFAINNTKSFEDIHQYREQIKRVKDSDDVPMVLVGNKCDLAARTVESRQAQDLARSYGIPYIETSAKTRQGVEDAFYTLVREIRQHKLRKLNPPDESGPGCMSCKCVLS",  # KRAS
    "MAQTGTEKTPEGPQPDRVQKFPGDTASQRITFASDGASIKGLQHPIRYLEEGEFFMAFKERFGDVNFVIGKDTLFTCKDEKVLQNFPTVVPDARRKPVSPFEGAPSITCFHCSTGFSWFAQSVTGKSHIQPGTSWDQTWEPKRNLKCRCGQEYESEGSRCKCCVDKTPKPQPAVRPTGRAFCSSHSTPGAKFQSSAVCGPHLQAKKEDNSKTMTRVTRKAAHELGRPLTPDSATVEKRKAKKTEECKSPAKNGDSFWAKDAKLDNVDLKDWEDMTDYIVDQKKELEKYCNTKCPKCKVFGPAILNRILKCKVIILAPAPDEEMITFQVEHHLDKKKFHDTLKKLGNINVFCAPVDIEEGVVVVDPLNSTSTNDPEVFKFCNFRKKSWFGKPTFSHTVPKYKHYIEVPEEPHVVLSLEPQAIQEGFKSLSDQAKDLLDQVNVYCPVLAETYKKMGVPDVIEVSPNKEPVCAVCGESIPTNIMLFCNDCGIYCCRKCQLEHLEGDKKRRGKKVESENSPVEKKKEPNKEAMQEEQQHSFLNRSRRRKLTKYVQDSQYDAHMKEKDKRWRLSQNKKDKGIRKLLDLEKEPSKEDTVKLDLTEKFRLSWTVVKDKALFHSIYEVEEIDGIGVLTLNQGADPDSLDQLKQKFQMLSKLPSNPADDPFVKIAELLGDNHLDLRRRQNLRNIKIGEFYLERKRTISPEDFIKTLTKGTFVTRSAYDNVIDATICDTCSQKSTTIDKPDRTPLKKGKLASKKPKPVKTKKGKKTVRTHSKTKSKKVPPKKRRDPNMNASIVGSDRGNVEVERPSTKTLKLSIQKLKDPVNPKPPPPPKTKTTPTPSQHINQNQVQEPTTKQQQESTTACERRRRETPTPTPPQAPPPPTQVNPGAQKNGLTPPQKKAKTAQKKPTTKKTVKKKQRLNKIISKVAELVHKKNAAKRSKLGKKVRKAKQSKKKDSEGASVNLTKDCNGVLMKSKLTKSRDLDRRKEKDLKDREKTKEKDLKDREKTKEKEPSKTSKVLKKDLTKPSIMSKKAKRKIKTTKGKVKSGRKGSKGDTSTKKKVKAKKPTKKSVKRSLAPETDEFDLEKDRKSLEESSNFEIEELPEQPNDLTELDIIDELSSYLDEPVDLEDYDSDEETSATSDSDSDSEYTKTSEVDLGSAVDVEGSAVDVEGSAVDVEGSSEVDRSDSEVETTTENTVETDRSENETAPKKKKKK",  # PARP1
    "MDVFMKGLSKAKEGVVAAAEKTKQGVAEAAGKTKEGVLYVGSKTKEGVVHGVATVAEKTKEQVTNVGGAVVTGVTAVAQKTVEGAGSIAAATGFVKKDQLGKNEEGAPQEGILEDMPVDPDNEAYEMPSEEGYQDYEPEA"  # Alpha-synuclein
]

# Create a simple h5torch file for demonstration
def create_demo_h5torch_file(file_name: str, sequences: List[str], overwrite: bool = False):
    """Create a demonstration h5torch file with proteins"""
    file_path = PROCESSED_DIR / file_name
    
    if file_path.exists() and not overwrite:
        print(f"File {file_path} already exists. Not overwriting.")
        return
    
    idx = np.arange(len(sequences))
    sequences = np.array(sequences)
    
    # Create the h5torch file
    f = h5torch.File(file_path, "w")
    
    # Register the ids as the central object
    f.register(idx, "central")
    
    # Register protein sequences as metadata aligned with axis 0
    f.register(sequences, 0, name="protein_sequences", dtype_save="bytes", dtype_load="str")
    
    # Create some dummy labels for the proteins
    protein_labels = np.random.rand(len(sequences))
    f.register(protein_labels, 0, name="y")
    
    f.close()
    print(f"Created demo h5torch file at {file_path}")
    
    return file_path

# Create the demo file
h5torch_file = "protein_embeddings_demo.h5torch"
create_demo_h5torch_file(h5torch_file, EXAMPLE_PROTEINS, overwrite=True)

# Add ESM embeddings
print("\n=== Generating ESM embeddings ===")
add_embeddings_to_h5torch(
    h5torch_file_name=h5torch_file,
    repo_name="ESM",
    entity_axis=0,
    feature_name="esm_embeddings",
    sequences=EXAMPLE_PROTEINS,
    use_batch=True,
    overwrite=True
)

# Add ESPF fingerprints
print("\n=== Generating ESPF fingerprints ===")
add_embeddings_to_h5torch(
    h5torch_file_name=h5torch_file,
    repo_name="ESPF",
    entity_axis=0,
    feature_name="espf_fingerprints",
    sequences=EXAMPLE_PROTEINS,
    use_batch=False,
    overwrite=True
)

# Print the h5torch file structure to verify
f = h5torch.File(PROCESSED_DIR / h5torch_file, "r")
print("\n=== H5Torch File Structure ===")
print(f"Available features: {list(f[1].keys())}")

# Print ESM embedding shape
if "esm_embeddings" in f[1]:
    print(f"ESM embeddings shape: {f[1]['esm_embeddings'].shape}")

# Print ESPF fingerprint shape
if "espf_fingerprints" in f[1]:
    print(f"ESPF fingerprints shape: {f[1]['espf_fingerprints'].shape}")

f.close()

Created demo h5torch file at data/processed/protein_embeddings_demo.h5torch

=== Generating ESM embeddings ===
Generating embeddings for 3 sequences using ESM...
Running embedding command: /home/robsyc/Desktop/thesis/MB-VAE-DTI/external/run_embeddings.sh ESM /home/robsyc/Desktop/thesis/MB-VAE-DTI/external/temp/esm_input_1743971616.txt /home/robsyc/Desktop/thesis/MB-VAE-DTI/external/temp/esm_output_1743971616.npy --batch
Error running embedding script: Command '['/home/robsyc/Desktop/thesis/MB-VAE-DTI/external/run_embeddings.sh', 'ESM', '/home/robsyc/Desktop/thesis/MB-VAE-DTI/external/temp/esm_input_1743971616.txt', '/home/robsyc/Desktop/thesis/MB-VAE-DTI/external/temp/esm_output_1743971616.npy', '--batch']' returned non-zero exit status 2.
STDOUT: Activating virtual environment at /home/robsyc/Desktop/thesis/MB-VAE-DTI/external/ESM/venv
Running: python /home/robsyc/Desktop/thesis/MB-VAE-DTI/external/ESM/script.py --input "/home/robsyc/Desktop/thesis/MB-VAE-DTI/external/temp/esm_input_1

CalledProcessError: Command '['/home/robsyc/Desktop/thesis/MB-VAE-DTI/external/run_embeddings.sh', 'ESM', '/home/robsyc/Desktop/thesis/MB-VAE-DTI/external/temp/esm_input_1743971616.txt', '/home/robsyc/Desktop/thesis/MB-VAE-DTI/external/temp/esm_output_1743971616.npy', '--batch']' returned non-zero exit status 2.

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
(4170,)


In [7]:
from mb_vae_dti.processing.embed_helper import get_target_fingerprint

aa_fp = get_target_fingerprint(aa)
print(aa_fp.shape)
print(aa_fp[:20])

(4170,)
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [9]:
from mb_vae_dti.processing.embed_helper import get_target_fingerprint

# Example 1: For a function that can process batches:
def batch_target_fingerprint(sequences):
    """Process multiple sequences at once"""
    if isinstance(sequences, str):
        return get_target_fingerprint(sequences)
    else:
        # Process a batch of sequences
        return [get_target_fingerprint(seq) for seq in sequences]

# Using batch processing (process 32 sequences at a time)
add_processed_feature(
    file_path="data/processed/data.h5torch",
    entity_path="1/Target_AA",
    process_func=batch_target_fingerprint,
    feature_name="Target_fp",
    batch_size=32,
    overwrite=True
)

# Example 2: If you had a drug fingerprint function
from mb_vae_dti.processing.embed_helper import get_drug_fingerprint

add_processed_feature(
    file_path="data/processed/data.h5torch",
    entity_path="0/Drug_SMILES",
    process_func=get_drug_fingerprint,
    feature_name="Drug_fp"
)

Processing 2047 entities with batch size 32, output dimension (4170,)


Processing Target_fp batches: 100%|██████████| 64/64 [00:15<00:00,  4.08it/s]


Successfully added Target_fp to data/processed/data.h5torch
Processing 149962 entities with batch size 1, output dimension (2048,)


Processing Drug_fp batches: 100%|██████████| 149962/149962 [02:12<00:00, 1135.22it/s]


Successfully added Drug_fp to data/processed/data.h5torch


---
---

In [6]:
from mb_vae_dti.processing.embed_helper import esm_batch_embed

In [7]:
aa1 = test_davis_metz[153]['1/Target_AA']
aa2 = test_davis_metz[13]['1/Target_AA']

In [8]:
len(aa1), len(aa2)

(491, 765)

In [9]:
result = esm_batch_embed([aa1, aa2])

torch.Size([1, 1, 493, 2560])
torch.Size([1, 1, 2560])
torch.Size([1, 1, 767, 2560])
torch.Size([1, 1, 2560])


In [10]:
result[0]

array([-0.00486905,  0.95842147, -0.0560879 , ..., -0.53510505,
       -0.0018255 ,  0.4314135 ], dtype=float32)

In [11]:
result[1].shape

(2560,)